In [8]:
import tweetnlp
model = tweetnlp.load_model('irony')

In [9]:
result = model.irony('Wow... soooo cool', return_probability=True)
type(result['probability']['irony'])

float

In [10]:
# taken from gpt.ipynb file
def calcFScore(truepos, falsepos, falseneg):
    FScoreResults = {}
    FScoreResults['precision'] = truepos/(truepos + falsepos)
    FScoreResults['recall'] = truepos/(truepos + falseneg)
    FScoreResults['F1'] = (2 * FScoreResults['precision'] * FScoreResults['recall'])/(FScoreResults['precision'] + FScoreResults['recall'])
    return FScoreResults

In [11]:
# Calculate the scores based on the answers from pysentimiento, this method is simply adapted from the other scores methods for GPT
def scoresSentimiento(dataset, resultColumn):
    truepos = 0
    falsepos = 0
    trueneg = 0
    falseneg = 0

    for index, row in dataset.iterrows():
        sentimientoEval = row[resultColumn]
        if (int(row[1]) == 1):
            if (int(sentimientoEval) == 1):
                truepos = truepos + 1
            else:
                falseneg = falseneg + 1
        elif (int(row[1]) == 0):
            if (int(sentimientoEval) == 1):
                falsepos = falsepos + 1
            else:
                trueneg = trueneg + 1

    numResults = calcFScore(truepos, falsepos, falseneg)
    numResults['tp'] = truepos
    numResults['fp'] = falsepos
    numResults['fn'] = falseneg
    numResults['tn'] = trueneg
    return numResults

In [13]:
import pandas as pd

datasetName = "tweet_eval_irony_test"
datasetPath = "datasets\\" + datasetName + ".csv"
data = pd.read_csv(datasetPath)
data = data.head(100)

results = []

for index, row in data.iterrows():
    result = model.irony(row[0], return_probability=True)
    if(result['probability']['irony'] >= 0.5):
        results.append(1)
    else:
        results.append(0)

data['result'] = results
scores = scoresSentimiento(data, 2)
scores

{'precision': 0.7,
 'recall': 0.5,
 'F1': 0.5833333333333334,
 'tp': 21,
 'fp': 9,
 'fn': 21,
 'tn': 49}